# SK MCP Server

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [ ]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.SemanticKernel, 1.49.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.49.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.49.0"
#r "nuget: ModelContextProtocol, 0.2.0-preview.1"
#r "nuget: dotenv.net"

using System;

using System.IO;
using System.Net.Http;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Agents;
using ModelContextProtocol.Client;

using dotenv.net;


#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003
#pragma warning disable SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025
#pragma warning disable SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,
#pragma warning disable SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102

### Start the Github MCP Server

In [ ]:
var projectPath = @"/home/alex/github/msalemor/sk-dev-training/notebooks/mcp/mcp1/server";

var mcpClient = await McpClientFactory.CreateAsync(
    new StdioClientTransport(new () {
        Name = "GitHub",
        Command = "npx",
        Arguments = ["@modelcontextprotocol/server-github"],
    })
);

### List all available tools in the MCP server

In [ ]:
var tools = await mcpClient.ListToolsAsync().ConfigureAwait(false);
foreach (var tool in tools)
{
    Console.WriteLine($"{tool.Name}: {tool.Description}");
}

### Read the API Key and endpoints from environment variables or the .env file

In [ ]:
// Load the .env file
DotEnv.Load();

// Get the OpenAI deployment name, endpoint, and key from the environment variables
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");

### Configure the Kernel and memory

In [ ]:
// Note: Added this because I am having problems with SSL certificate validation
var handler = new HttpClientHandler();
handler.CheckCertificateRevocationList = false;
var httpClient = new HttpClient(handler);

In [ ]:
#pragma warning disable SKEXP0010
var kernelBuilder = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: deploymentName,
        endpoint: endpoint,
        apiKey: apiKey,
        httpClient: httpClient);
Kernel kernel = kernelBuilder.Build();

### Add the MCP server as an SK plugin and use it automatically

In [ ]:
#pragma warning disable SKEXP0001
kernel.Plugins.AddFromFunctions("GitHub", tools.Select(aiFunction => aiFunction.AsKernelFunction()));

#pragma warning disable SKEXP0001
AzureOpenAIPromptExecutionSettings executionSettings = new()
{
    Temperature = 0,
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(options: new() { RetainArgumentTypes = true })
};

### Test the MCP server in a prompt

In [ ]:
// Test using GitHub tools
var prompt = "Summarize the last four commits to the microsoft/semantic-kernel repository?";
var result = await kernel.InvokePromptAsync(prompt, new(executionSettings)).ConfigureAwait(false);
Console.WriteLine($"\n\n{prompt}\n{result}");